###  Features ever tried
full_data is created in /features/final/features_final.ipynb file

#### Half bathrooms

In [86]:
full_data['half_bathrooms'] = ((np.round(full_data.bathrooms) - full_data.bathrooms)!=0).astype(float) # Half bathrooms? 1.5, 2.5, 3.5...

#### Duplicate number

In [87]:
# number of duplicates, boolean for duplicates 
def duplicate(X,columns):
    dup_data = X[X.duplicated(columns,keep=False)]
    dup_data = dup_data.sort_values(columns,ascending=[True]*len(columns))
    keep = dup_data.drop_duplicates(columns,keep='first')
    
    X['has_dup'] = [0]*X.shape[0]
    X['num_dup'] = [1]*X.shape[0]
    
    for i in range(keep.shape[0]):
        df = keep.iloc[i]
        dup_df = X[(X[columns] == df[columns]).sum(axis=1) == len(columns)]
        X.loc[dup_df.index,'has_dup'] = [1]*dup_df.shape[0]
        X.loc[dup_df.index,'num_dup'] = [dup_df.shape[0]]*dup_df.shape[0]
        
    return X

In [88]:
full_data = duplicate(full_data,['price', 'latitude', 'longitude', 'bathrooms','bedrooms', 'street_address', 'building_id'])

#### Address distance

In [ ]:
full_data['address_distance'] = [editdistance.eval(x.lower(),y.lower()) for x,y in zip(np.array(full_data.display_address),np.array(full_data.street_address))]

#### Location based price level

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=300)
price_df = pd.concat([full_data['bedrooms'],full_data['bathrooms'],full_data['latitude'],full_data['longitude'],full_data['price']], axis=1)
model.fit(price_df.drop(['price'], axis=1), price_df['price'])
pred_price = model.predict(price_df.drop(['price'], axis=1))
price_df['predicted_price'] = pd.DataFrame(pred_price, columns=['predicted_price'])
price_df['pred_price_ratio'] = price_df['price'] / price_df['predicted_price']
full_data['pred_price_ratio'] = price_df['pred_price_ratio']

#### Price per sqft

In [ ]:
A = 1
B = 0.5
C = [1,4]
D = [0,2]
full_data['price_per_sqft'] = (full_data.price / (A + full_data.bedrooms.clip(C[0], C[1]) + B*full_data.bathrooms.clip(D[0], D[1]))).values

#### Manager Skill

In [ ]:
# functions to compute interest level fraction and manager skill
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

class manager_skill(BaseEstimator, TransformerMixin):
    
    def __init__(self, threshold = 5):
        
        self.threshold = threshold
        
    def _reset(self):
     
        if hasattr(self, 'mapping_'):
            
            self.mapping_ = {}
            self.mean_skill_ = 0.0
            
    def fit(self, X,y):
    
        self._reset()
        
        temp = pd.concat([X.manager_id,pd.get_dummies(y)], axis = 1).groupby('manager_id').mean()
        temp.columns = ['low_frac', 'medium_frac', 'high_frac']
        temp['count'] = X.groupby('manager_id').count().iloc[:,1]
        
        print(temp.head())

        mean = temp.loc[temp['count'] >= self.threshold, ['low_frac', 'medium_frac', 'high_frac']].mean()
        
        temp.loc[temp['count'] < self.threshold, ['low_frac', 'medium_frac', 'high_frac']] = mean
        
        self.mapping_ = temp[['low_frac', 'medium_frac', 'high_frac']]
        self.mean_skill_ = mean
            
        return self
    
    def transform(self, X):

        X = pd.merge(left = X, right = self.mapping_, how = 'left', left_on = 'manager_id', right_index = True)
        X[['low_frac', 'medium_frac', 'high_frac']] = X[['low_frac', 'medium_frac', 'high_frac']].fillna(self.mean_skill_)
        
        return X

In [ ]:
train_data = full_data.iloc[0:train_data.shape[0]]
test_data = full_data.iloc[train_data.shape[0]:full_data.shape[0]]

In [ ]:
[train_data.shape[0],test_data.shape[0]]

In [ ]:
# assign interest fraction on test dataset
trans = manager_skill()
trans.fit(train_data, train_data['interest_level'])
train_data = trans.transform(train_data)
train_data.head()
test_data = trans.transform(test_data)
test_data.head()

In [ ]:
full_data=pd.concat([train_data
                       ,test_data])

### Create training and testing data

In [73]:
%%time
##Baseline with features from "features" and street address

full_vars = num_vars + date_num_vars + additional_num_vars + interactive_num_vars + LE_vars + hcc_vars + num_cat_vars + img_vars # + additioanl_vars

train_x = sparse.hstack([full_data[full_vars], feature_sparse,st_addr_sparse]).tocsr()[:train_size]
train_y = full_data['target'][:train_size].values

test_x = sparse.hstack([full_data[full_vars], feature_sparse,st_addr_sparse]).tocsr()[train_size:]
test_y = full_data['target'][train_size:].values


full_vars = full_vars + feature_vars + st_addr_vars
print ("training data size: ", train_x.shape,"testing data size: ", test_x.shape)

('training data size: ', (49352, 261), 'testing data size: ', (74659, 261))
Wall time: 3.59 s
